In [2]:
# import libraries

from bs4 import BeautifulSoup  ## web scraping
import requests ## request html for a page(s)
import csv ## read or write to csv
import pandas as pd ## pandas to work with data

In [10]:
## url here is not a copy and paste. look at templating {}
url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html"

In [13]:
## Change the value of total_pages to match number of pages on site.
total_pages = 5

In [14]:
for link in [url.format(page) for page in range(1,total_pages)]:
    print(link)
    site = requests.get(link)
    print(site.status_code)

https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html
200
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page2.html
200
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page3.html
200
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page4.html
200


In [17]:
## imgage of 6 urls

In [53]:
base_link = "http://www.example{}.html"

In [54]:
total_pages = 7

In [55]:
all_urls = []
for url_number in range(1,total_pages):
    print(base_link.format(url_number))
    all_urls.append(base_link.format(url_number))
    
all_urls


http://www.example1.html
http://www.example2.html
http://www.example3.html
http://www.example4.html
http://www.example5.html
http://www.example6.html


['http://www.example1.html',
 'http://www.example2.html',
 'http://www.example3.html',
 'http://www.example4.html',
 'http://www.example5.html',
 'http://www.example6.html']

In [56]:
all_urls_lc = [base_link.format(url_number) for url_number in range(1,total_pages)]
all_urls_lc

['http://www.example1.html',
 'http://www.example2.html',
 'http://www.example3.html',
 'http://www.example4.html',
 'http://www.example5.html',
 'http://www.example6.html']